# Study on Decentraland 

### Retrieve data from decentraland

In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm
from time import sleep

from scipy.interpolate import griddata

import matplotlib.pyplot as plt
import seaborn as sns

import requests

pd.options.display.max_columns = None

In [2]:
url_decentraland = "https://api.decentraland.org/"
response_tiles = requests.get(url_decentraland + "v2/tiles")
tiles = pd.DataFrame.from_dict(response_tiles.json()["data"], orient="index").reset_index(drop=True)
tiles = tiles.rename(columns={"id":"xy"})[["xy","x","y","type","estateId","owner","tokenId","name","price"]]

today = "220126"
tiles.to_csv('Y:\decentraland_data_retrieve/tiles/tile_info_{date}.csv'.format(date=today), header=True)

In [3]:
print(len(tiles))
print(tiles['x'].max(), tiles['y'].max())
print(tiles['x'].min(), tiles['y'].min())

92598
163 158
-150 -150


In [4]:
tiles.head(3)

,xy,x,y,type,estateId,owner,tokenId,name,price
0,"-150,150",-150,150,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN
1,"-150,149",-150,149,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN
2,"-150,148",-150,148,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN


#### Private parcel owner

In [5]:
owner_count = pd.DataFrame(tiles[tiles["type"]=='owned']['owner'].value_counts().rename_axis('owner_address').reset_index(name='counts'))

In [6]:
len(owner_count)

6127

In [7]:
owner_count.head(3)

,owner_address,counts
0,0x1463b7162103247c5d464f104f7c9da61dea1bfc,3548
1,0xb096e691b197e4622bc0033a2efda76082b33783,1527
2,0x5326a48a8badfbbf2fb6c4a03ae6685919e3040f,1397


## Etherscan

#### Common Things

In [8]:
module = "account"
address = "0xb8b7dc5b439f72154dafc2c13376d861e0c19513"
API_KEY_ETH = "UMSMKQKRPN8PGYIR1Q9HGVE561S393JI5V"

##### Normal Transaction

#### ERC 20 Transaction

In [25]:
from pandas import json_normalize

module = "account"
API_KEY_ETH = "UMSMKQKRPN8PGYIR1Q9HGVE561S393JI5V"

for owner in tqdm(owner_count["owner_address"]):
   address = owner

   tx_20_action = "tokentx"
   url_20_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act=tx_20_action, adrs=address, api=API_KEY_ETH)
   response_20_tx = requests.get(url_20_tx)
   try:
       df_20_tx = json_normalize(response_20_tx.json()["result"])
       df_20_tx_col = df_20_tx[["timeStamp", "hash", "from", "contractAddress", "to", "value", "tokenName", "tokenSymbol", "tokenDecimal", "transactionIndex", "gas", "gasPrice"]]
   except:
       pass
   df_20_tx_col.to_csv("Y:\decentraland_data_retrieve/tx_erc20/erc20_{owner}.csv".format(owner=owner), header=True)

print("ERC20 data retrieve done!")

100%|██████████| 6090/6090 [2:00:40<00:00,  1.19s/it]  

ERC20 data retrieve done!


In [24]:
from pandas import json_normalize

module = "account"
API_KEY_ETH = "UMSMKQKRPN8PGYIR1Q9HGVE561S393JI5V"

for owner in tqdm(owner_count["owner_address"]):
   address = owner

   # Get ERC20 txs & save to csv file
   tx_20_action = "tokentx"
   url_20_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act=tx_20_action, adrs=address, api=API_KEY_ETH)
   response_20_tx = requests.get(url_20_tx)
   
   df_20_tx = json_normalize(response_20_tx.json()["result"])
# df_20_tx = pd.DataFrame.from_dict(response_20_tx.json()["result"], orient="columns")
   try:
        df_20_tx_col = df_20_tx[["timeStamp", "hash", "from", "contractAddress", "to", "value", "tokenName", "tokenSymbol", "tokenDecimal", "transactionIndex", "gas", "gasPrice"]]
   except:
       pass
   df_20_tx_col.to_csv("Y:\decentraland_data_retrieve/tx_erc20/erc20_{owner}.csv".format(owner=owner), header=True)

print("ERC20 data retrieve done!")

 96%|█████████▋| 5870/6090 [1:56:26<04:21,  1.19s/it]  


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
from pandas import json_normalize
df = json_normalize(response_20_tx.json()['result'])

In [20]:
type(response_20_tx.json()['result'])

list